# SA_SLM 추론 노트북

## 필요 파일
- `sa_slm_adapter.zip`: 학습 노트북에서 다운로드한 파일

## 실행 전
런타임 > 런타임 유형 변경 > **GPU (T4)** 선택

In [ ]:
# 1. 환경 설정
!pip install -q torch transformers accelerate peft bitsandbytes

In [ ]:
# 2. Adapter 업로드
from google.colab import files
import zipfile, os

print('sa_slm_adapter.zip 파일을 업로드하세요')
uploaded = files.upload()

for f in uploaded:
    if f.endswith('.zip'):
        zipfile.ZipFile(f).extractall('.')
        print(f'{f} 압축 해제 완료')

print('OK' if os.path.exists('./adapter') else 'adapter 폴더 없음')

In [ ]:
# 3. 모델 로드
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

MODEL = 'Qwen/Qwen2.5-3B-Instruct'

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL, quantization_config=bnb, device_map='auto', trust_remote_code=True
)
model = PeftModel.from_pretrained(model, './adapter')
model.eval()

print('Model loaded!')

In [ ]:
# 4. 추론 함수
def ask(prompt):
    msgs = [
        {'role': 'system', 'content': '생기부 설계 전문가. 성공 사례 기반 차별화된 활동 추천.'},
        {'role': 'user', 'content': prompt}
    ]
    txt = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    inp = tokenizer(txt, return_tensors='pt').to(model.device)
    
    with torch.no_grad():
        out = model.generate(**inp, max_new_tokens=800, temperature=0.7, do_sample=True)
    
    return tokenizer.decode(out[0][inp['input_ids'].shape[1]:], skip_special_tokens=True)

print('ask() 함수 준비 완료')

In [ ]:
# 5. 활동 추천 테스트
profile = '''계열: 자연
성적: 3등급대
관심: 인공지능, 데이터사이언스
가치관: AI 윤리, 디지털 격차 해소
목표: 컴퓨터공학'''

print(ask('차별화된 활동을 추천하세요.\n\n' + profile))

In [ ]:
# 6. 세특 문장 생성
activity = '''과목: 정보
활동: 이미지 분류 데이터셋 클래스 불균형 문제 탐구. 언더/오버샘플링 직접 구현하여 비교 실험.'''

print(ask('NEIS 세특 문장으로 작성. 3인칭 서술체.\n\n' + activity))

In [ ]:
# 7. 역량 평가
statement = '''문장: 코딩 동아리에서 프로그래밍을 배우고 간단한 프로젝트를 수행함.
전공: 컴퓨터공학'''

print(ask('역량 평가 및 보완 제안.\n\n' + statement))

In [ ]:
# 8. 직접 테스트
my_profile = '''계열: 인문
성적: 2등급대
관심: 경영학, 마케팅
가치관: ESG, 사회적 기업
목표: 경영학'''

print(ask('차별화된 활동을 추천하세요.\n\n' + my_profile))